In [1]:
import requests
import pandas as pd
import time
#1.全国各省实时数据爬取
#2.世界各国实时数据爬取
#3.全国历史数据爬取
#4.世界各国历史数据爬取

In [2]:
#设置请求头，伪装为浏览器
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'
}

In [3]:
url = 'https://c.m.163.com/ug/api/wuhan/app/data/list-total'   # 定义要访问的地址 
r = requests.get(url, headers=headers)  # 使用requests发起请求

In [4]:
print(r.status_code)  # 查看请求状态

200


In [5]:
#由于字符串格式不方便进行分析，并且在网页预览中发现数据为类似字典的json格式，所以我们将其转为json格式。
import json
data_json = json.loads(r.text)

In [6]:
data_json.keys()

dict_keys(['reqId', 'code', 'msg', 'data', 'timestamp'])

In [7]:
data = data_json['data'] # 取出json中的数据
data.keys()

dict_keys(['chinaTotal', 'chinaDayList', 'lastUpdateTime', 'overseaLastUpdateTime', 'areaTree'])

In [8]:
#1.全国各省实时数据爬取

data_province = data['areaTree'][2]['children']  # 取出中国各省的实时数据children

In [9]:
data_province

[{'today': {'confirm': 1,
   'suspect': None,
   'heal': 173,
   'dead': 6,
   'severe': None,
   'storeConfirm': None,
   'input': 0},
  'total': {'confirm': 67802,
   'suspect': 0,
   'heal': 63326,
   'dead': 3193,
   'severe': 0,
   'input': 0},
  'extData': {},
  'name': '湖北',
  'id': '420000',
  'lastUpdateTime': '2020-04-01 13:28:00',
  'children': [{'today': {'confirm': 0,
     'suspect': 0,
     'heal': 173,
     'dead': 5,
     'severe': 0,
     'storeConfirm': None},
    'total': {'confirm': 50006,
     'suspect': 0,
     'heal': 46175,
     'dead': 2553,
     'severe': 0},
    'extData': {'confirmNote': '',
     'suspectNote': '',
     'healNote': '',
     'deadNote': '',
     'incrConfirmNote': '',
     'incrSevereNote': ''},
    'name': '武汉',
    'id': '420100',
    'lastUpdateTime': '2020-04-01 13:28:00',
    'children': []},
   {'today': {'confirm': 0,
     'suspect': None,
     'heal': 0,
     'dead': 0,
     'severe': None,
     'storeConfirm': None},
    'total': {'c

In [ ]:
data_province[0].keys() # 查看每个省键名称

In [ ]:
for i in range(len(data_province)): # 遍历查看各省名称、更新时间
    print(data_province[i]['name'],data_province[i]['lastUpdateTime'])
    if i == 5:
        break

In [ ]:
# 获取id、lastUpdateTime、name
info = pd.DataFrame(data_province)[['id','lastUpdateTime','name']]
info.head()

In [ ]:
# 列表推导式获取today中的数据
today_data = pd.DataFrame([province['today'] for province in data_province ]) 
today_data.head()

In [ ]:
['today_'+i for i in today_data.columns]

In [ ]:
today_data.columns = ['today_'+i for i in today_data.columns] # 由于today中键名和total键名相同，因此需要修改列名称
today_data.head()

In [ ]:
# 获取total中的数据
total_data = pd.DataFrame([province['total'] for province in data_province ])
total_data.columns = ['total_'+i for i in total_data.columns]
total_data.head()

In [ ]:
pd.concat([info,total_data,today_data],axis=1).head() # 将total、today和省份信息三个数据合并

In [10]:
#总结前面的方法
# 将提取数据的方法封装为函数
def get_data(data,info_list):
    info = pd.DataFrame(data)[info_list] # 主要信息
    
    today_data = pd.DataFrame([i['today'] for i in data ])       # 提取today的数据
    today_data.columns = ['today_'+i for i in today_data.columns] # 修改列名 columns
    
    total_data = pd.DataFrame([i['total'] for i in data ])       # 提取total的数据
    total_data.columns = ['total_'+i for i in total_data.columns] # 修改列名 columns
    
    return pd.concat([info,total_data,today_data],axis=1) # info、today和total横向合并最终得到汇总的数据

In [11]:
today_province = get_data(data_province,['id','lastUpdateTime','name'])
today_province.head()

,id,lastUpdateTime,name,total_confirm,total_suspect,total_heal,total_dead,total_severe,total_input,today_confirm,today_suspect,today_heal,today_dead,today_severe,today_storeConfirm,today_input
0,420000,2020-04-01 13:28:00,湖北,67802,0,63326,3193,0,0,1.0,NaN,173.0,6.0,NaN,None,0.0
1,440000,2020-04-01 13:11:48,广东,1501,0,1357,8,0,0,11.0,NaN,4.0,0.0,NaN,None,0.0
2,410000,2020-04-01 13:50:22,河南,1276,0,1251,22,0,0,0.0,NaN,0.0,0.0,NaN,None,0.0
3,330000,2020-04-01 09:03:23,浙江,1257,0,1226,1,0,0,0.0,0.0,0.0,0.0,0.0,None,0.0
4,430000,2020-04-01 11:01:44,湖南,1018,0,1014,4,0,0,0.0,NaN,0.0,0.0,NaN,None,0.0


In [12]:
def save_data(data,name): # 定义保存数据方法
    file_name = name+'_'+time.strftime('%Y_%m_%d',time.localtime(time.time()))+'.csv'#加上操作的时间戳信息
    data.to_csv(file_name,index=None,encoding='utf_8_sig')
    print(file_name+' 保存成功！')
save_data(today_province,'today_province')

today_province_2020_04_01.csv 保存成功！


In [13]:
#2.世界各国实时数据爬取
areaTree = data['areaTree'] # 取出areaTree
areaTree[0] # 查看第一个国家的数据

{'today': {'confirm': 32,
  'suspect': 0,
  'heal': 0,
  'dead': 2,
  'severe': 0,
  'storeConfirm': None},
 'total': {'confirm': 394, 'suspect': 0, 'heal': 3, 'dead': 10, 'severe': 0},
 'extData': {'confirmNote': '',
  'suspectNote': '',
  'healNote': '',
  'deadNote': '',
  'incrConfirmNote': '',
  'incrSevereNote': ''},
 'name': '突尼斯',
 'id': '9577772',
 'lastUpdateTime': '2020-04-01 07:38:32',
 'children': []}

In [14]:
for i in range(len(areaTree)):  # 查看各国家名称和更新时间
    print(areaTree[i]['name'],areaTree[i]['lastUpdateTime'])
    if i == 5:#取前五行
        break

突尼斯 2020-04-01 07:38:32
塞尔维亚 2020-04-01 12:21:07
中国 2020-04-01 14:31:31
日本 2020-04-01 14:02:32
泰国 2020-04-01 14:02:04
新加坡 2020-04-01 00:00:31


In [15]:
today_world = get_data(areaTree,['id','lastUpdateTime','name']) # areaTree
today_world

,id,lastUpdateTime,name,total_confirm,total_suspect,total_heal,total_dead,total_severe,total_input,today_confirm,today_suspect,today_heal,today_dead,today_severe,today_storeConfirm,today_input
0,9577772,2020-04-01 07:38:32,突尼斯,394,0,3,10,0,NaN,32.0,0.0,0.0,2.0,0.0,None,NaN
1,9507896,2020-04-01 12:21:07,塞尔维亚,900,0,0,23,0,NaN,115.0,0.0,0.0,7.0,0.0,None,NaN
2,0,2020-04-01 14:31:31,中国,82640,183,76417,3321,466,806.0,95.0,0.0,192.0,7.0,-62.0,None,35.0
3,1,2020-04-01 14:02:32,日本,2945,0,1043,77,0,NaN,22.0,NaN,0.0,0.0,NaN,None,NaN
4,2,2020-04-01 14:02:04,泰国,1771,0,127,12,0,NaN,120.0,0.0,0.0,2.0,0.0,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,82333,2020-03-27 11:33:37,格恩西岛,1,0,0,0,0,NaN,NaN,0.0,NaN,NaN,0.0,None,NaN
196,95677,2020-04-01 10:54:37,罗马尼亚,2245,0,220,69,0,NaN,136.0,0.0,11.0,4.0,0.0,None,NaN
197,95436,2020-04-01 08:31:03,阿曼,192,0,34,1,0,NaN,13.0,0.0,5.0,1.0,0.0,None,NaN
198,95672,2020-04-01 00:00:31,列支敦士登,64,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN


In [16]:
save_data(today_world,'today_world')

today_world_2020_04_01.csv 保存成功！


In [17]:
#3.全国历史数据爬取
chinaDayList = data['chinaDayList'] # 取出 chinaDayList
type(chinaDayList) # 查看chinaDayList的格式

list

In [18]:
chinaDayList[0]#可以看出一个省一天的的数据格式如上，这和之前的数据结构一样，因此可以使用之前的方法得到数据。

{'date': '2020-01-20',
 'today': {'confirm': 291,
  'suspect': 27,
  'heal': 25,
  'dead': 6,
  'severe': 0,
  'storeConfirm': None},
 'total': {'confirm': 291, 'suspect': 54, 'heal': 25, 'dead': 6, 'severe': 0},
 'lastUpdateTime': None}

In [19]:
alltime_China = get_data(chinaDayList,['date','lastUpdateTime'])  # 应用提取数据的方法get_data
alltime_China.head()
#通过上述方法得到了湖北省的历史数据

,date,lastUpdateTime,total_confirm,total_suspect,total_heal,total_dead,total_severe,today_confirm,today_suspect,today_heal,today_dead,today_severe,today_storeConfirm
0,2020-01-20,None,291,54,25,6,0,291,27,25,6,0,None
1,2020-01-21,None,440,37,25,9,102,149,26,0,3,0,None
2,2020-01-22,None,571,393,28,17,95,131,257,3,8,0,None
3,2020-01-23,None,830,1072,34,25,177,259,680,6,8,0,None
4,2020-01-24,None,1287,1965,38,41,237,457,1118,4,16,0,None


In [20]:
#下一步就是要获得每个省的历史数据
#在湖北省历史数据的地址中，我们发现参数aeraCode=420000，而这刚好和全国各省实时数据today_province中的id对应.
#我们发现id就是各省的行政代码
#由于网站上数据中并没有显示省的名称，因此需要写入每个省的名称
#为了便于写入各省的名称，我们需要生成一个各省行政代码和省名称对应的字典province_dict。
province_dict = { num:name for num,name in zip(today_province['id'],today_province['name']) } # zip
start = time.time()
for province_id in province_dict: # 遍历各省编号
    
    try:
        # 按照省编号访问每个省的数据地址，并获取json数据
        url = 'https://c.m.163.com/ug/api/wuhan/app/data/list-by-area-code?areaCode='+province_id  
        r = requests.get(url, headers=headers)
        data_json = json.loads(r.text)
        
        # 提取各省数据，然后写入各省名称
        province_data = get_data(data_json['data']['list'],['date'])
        province_data['name'] = province_dict[province_id]
        
        # 合并数据
        if province_id == '420000':
            alltime_province = province_data
        else:
            alltime_province = pd.concat([alltime_province,province_data]) 
            
        print('-'*20,province_dict[province_id],'成功',
              province_data.shape,alltime_province.shape,
              ',累计耗时:',round(time.time()-start),'-'*20)
        
        # 设置延迟等待
        time.sleep(10)
        
    except:
        print('-'*20,province_dict[province_id],'wrong','-'*20)

-------------------- 湖北 成功 (72, 13) (72, 13) ,累计耗时: 0 --------------------
-------------------- 广东 成功 (69, 13) (141, 13) ,累计耗时: 10 --------------------
-------------------- 河南 成功 (71, 13) (212, 13) ,累计耗时: 20 --------------------
-------------------- 浙江 成功 (68, 13) (280, 13) ,累计耗时: 31 --------------------
-------------------- 湖南 成功 (70, 13) (350, 13) ,累计耗时: 42 --------------------
-------------------- 安徽 成功 (69, 13) (419, 13) ,累计耗时: 52 --------------------
-------------------- 江西 成功 (69, 13) (488, 13) ,累计耗时: 64 --------------------
-------------------- 山东 成功 (70, 13) (558, 13) ,累计耗时: 74 --------------------
-------------------- 香港 成功 (71, 13) (629, 13) ,累计耗时: 84 --------------------
-------------------- 江苏 成功 (70, 13) (699, 13) ,累计耗时: 94 --------------------
-------------------- 北京 成功 (72, 13) (771, 13) ,累计耗时: 104 --------------------
-------------------- 重庆 成功 (71, 13) (842, 13) ,累计耗时: 114 --------------------
-------------------- 四川 成功 (71, 13) (913, 13) ,累计耗时: 125 -------------------

In [21]:
save_data(alltime_province,'alltime_province')

alltime_province_2020_04_01.csv 保存成功！


In [22]:
#4.世界各国历史数据爬取
#因为原始数据中没有国家名称，为了得到每个国家的名称，需要生成国家编号和国家名称的键值对，
#这样就可以存储国家名称，在之前的世界各国实时数据today_world中有国家的编号和名称，可以用它来生成键值对。
country_dict = { key:value for key,value in zip(today_world['id'], today_world['name'])}  
start = time.time()
for country_id in country_dict: # 遍历每个国家的编号
    
    try:
        # 按照编号访问每个国家的数据地址，并获取json数据
        url = 'https://c.m.163.com/ug/api/wuhan/app/data/list-by-area-code?areaCode='+country_id   #每个国家的数据地址
        r = requests.get(url, headers=headers)
        json_data = json.loads(r.text)
        
        # 生成每个国家的数据
        country_data = get_data(json_data['data']['list'],['date'])
        country_data['name'] = country_dict[country_id]

        # 数据叠加
        if country_id == '9577772':
            alltime_world = country_data
        else:
            alltime_world = pd.concat([alltime_world,country_data]) 
            
        print('-'*20,country_dict[country_id],'成功',country_data.shape,alltime_world.shape,
              ',累计耗时:',round(time.time()-start),'-'*20)
        
        time.sleep(10)

    except:
        print('-'*20,country_dict[country_id],'wrong','-'*20)

-------------------- 突尼斯 成功 (21, 13) (21, 13) ,累计耗时: 0 --------------------
-------------------- 塞尔维亚 成功 (22, 13) (43, 13) ,累计耗时: 10 --------------------
-------------------- 中国 成功 (57, 13) (100, 13) ,累计耗时: 20 --------------------
-------------------- 日本 成功 (60, 13) (160, 13) ,累计耗时: 31 --------------------
-------------------- 泰国 成功 (35, 13) (195, 13) ,累计耗时: 41 --------------------
-------------------- 新加坡 成功 (49, 13) (244, 13) ,累计耗时: 51 --------------------
-------------------- 韩国 成功 (72, 13) (316, 13) ,累计耗时: 61 --------------------
-------------------- 澳大利亚 成功 (37, 13) (353, 13) ,累计耗时: 72 --------------------
-------------------- 德国 成功 (39, 13) (392, 13) ,累计耗时: 82 --------------------
-------------------- 美国 成功 (41, 13) (433, 13) ,累计耗时: 92 --------------------
-------------------- 马来西亚 成功 (36, 13) (469, 13) ,累计耗时: 102 --------------------
-------------------- 越南 成功 (28, 13) (497, 13) ,累计耗时: 112 --------------------
-------------------- 圣巴泰勒米 成功 (1, 13) (498, 13) ,累计耗时: 122 ----------

-------------------- 乌干达 成功 (8, 13) (1815, 13) ,累计耗时: 1054 --------------------
-------------------- 厄立特里亚 成功 (6, 13) (1821, 13) ,累计耗时: 1064 --------------------
-------------------- 刚果（布） 成功 (4, 13) (1825, 13) ,累计耗时: 1074 --------------------
-------------------- 津巴布韦 成功 (8, 13) (1833, 13) ,累计耗时: 1084 --------------------
-------------------- 丹麦 成功 (34, 13) (1867, 13) ,累计耗时: 1094 --------------------
-------------------- 阿鲁巴 成功 (1, 13) (1868, 13) ,累计耗时: 1105 --------------------
-------------------- 斐济 成功 (2, 13) (1870, 13) ,累计耗时: 1115 --------------------
-------------------- 伯利兹 成功 (2, 13) (1872, 13) ,累计耗时: 1125 --------------------
-------------------- 缅甸 成功 (7, 13) (1879, 13) ,累计耗时: 1135 --------------------
-------------------- 塞浦路斯 成功 (20, 13) (1899, 13) ,累计耗时: 1145 --------------------
-------------------- 关岛 成功 (1, 13) (1900, 13) ,累计耗时: 1155 --------------------
-------------------- 科索沃 成功 (6, 13) (1906, 13) ,累计耗时: 1166 --------------------
-------------------- 吉尔吉斯斯坦 成功 (8, 1

In [23]:
save_data(alltime_world,'alltime_world')

alltime_world_2020_04_01.csv 保存成功！
